In [1]:
# Import our dependencies
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split as holdout
from sklearn.linear_model import LinearRegression

complete_data_df= pd.read_csv("resources/avg_by_region.csv")
complete_data_df.head()


,id,age,average_income,sex,bmi,children,smoker,region,charges,average_age
0,0,18,10753.5,male,33.77,1,no,southeast,1725.5523,78.43
1,1,18,10753.5,male,34.10,0,no,southeast,1137.0110,78.43
2,5,18,10753.5,male,31.68,2,yes,southeast,34303.1672,78.43
3,9,18,10753.5,female,36.85,0,yes,southeast,36149.4835,78.43
4,11,18,10753.5,female,38.28,0,no,southeast,1631.8212,78.43


In [2]:
complete_data_df.nunique()

id                1338
age                 47
average_income      47
sex                  2
bmi                548
children             6
smoker               2
region               4
charges           1337
average_age          4
dtype: int64

In [3]:
complete_data_cat = complete_data_df.dtypes[complete_data_df.dtypes == "object"].index.tolist()
complete_data_df

,id,age,average_income,sex,bmi,children,smoker,region,charges,average_age
0,0,18,10753.50,male,33.770,1,no,southeast,1725.55230,78.43
1,1,18,10753.50,male,34.100,0,no,southeast,1137.01100,78.43
2,5,18,10753.50,male,31.680,2,yes,southeast,34303.16720,78.43
3,9,18,10753.50,female,36.850,0,yes,southeast,36149.48350,78.43
4,11,18,10753.50,female,38.280,0,no,southeast,1631.82120,78.43
...,...,...,...,...,...,...,...,...,...,...
1333,1316,64,87985.91,male,24.700,1,no,northwest,30166.61817,79.88
1334,1321,64,87985.91,female,30.115,3,no,northwest,16455.70785,79.88
1335,1323,64,87985.91,female,32.965,0,no,northwest,14692.66935,79.88
1336,1330,64,87985.91,male,37.905,0,no,northwest,14210.53595,79.88


In [4]:
complete_data_df.head(5)

,id,age,average_income,sex,bmi,children,smoker,region,charges,average_age
0,0,18,10753.5,male,33.77,1,no,southeast,1725.5523,78.43
1,1,18,10753.5,male,34.10,0,no,southeast,1137.0110,78.43
2,5,18,10753.5,male,31.68,2,yes,southeast,34303.1672,78.43
3,9,18,10753.5,female,36.85,0,yes,southeast,36149.4835,78.43
4,11,18,10753.5,female,38.28,0,no,southeast,1631.8212,78.43


In [5]:
dummies = pd.get_dummies(complete_data_df[['sex', 'smoker', 'region']], drop_first=True)
text_features = pd.concat([complete_data_df.drop(['sex', 'smoker', 'region'],axis=1), dummies],axis=1)
text_features.head()

,id,age,average_income,bmi,children,charges,average_age,sex_male,smoker_yes,region_northwest,region_southeast,region_southwest
0,0,18,10753.5,33.77,1,1725.5523,78.43,1,0,0,1,0
1,1,18,10753.5,34.10,0,1137.0110,78.43,1,0,0,1,0
2,5,18,10753.5,31.68,2,34303.1672,78.43,1,1,0,1,0
3,9,18,10753.5,36.85,0,36149.4835,78.43,0,1,0,1,0
4,11,18,10753.5,38.28,0,1631.8212,78.43,0,0,0,1,0


In [6]:
x = text_features.drop(['id', 'charges', 'sex_male', 'region_northwest', 'region_southeast', 'region_southwest', "average_age"], axis = 1)
y = text_features.charges

# changing the number of degrees will reduce the predicted value. 2 is the optimal number of degrees 
pol = PolynomialFeatures (degree = 2)
x_pol = pol.fit_transform(x)

x_train, x_test, y_train, y_test = holdout(x_pol, y, test_size=0.2, random_state=0)
Pol_reg = LinearRegression()
Pol_reg.fit(x_train, y_train)
y_train_pred = Pol_reg.predict(x_train)
y_test_pred = Pol_reg.predict(x_test)
print(Pol_reg.intercept_)
print(Pol_reg.coef_)
print(Pol_reg.score(x_test, y_test))

-6509.09521486425
[ 0.00000000e+00  7.80050457e+01 -7.73257364e-03  4.66448570e+02
  2.03357914e+03 -9.58077742e+03 -3.98391644e+00  8.12913254e-03
  1.29693936e+00  3.78705571e+01 -1.36225354e+02 -2.58735740e-06
 -6.91335716e-04 -2.65229305e-02  7.40708458e-02 -7.36427000e+00
 -1.58700269e+01  1.45490822e+03 -1.58199141e+02 -7.58577010e+02
 -9.58077742e+03]
0.811710094612918


In [7]:
# pridicting the charges 
y_test_pred = Pol_reg.predict(x_test)

#compare the acutal value with our predicted value 
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_test_pred})
df

,Actual,Predicted
578,8627.54110,9147.980673
610,24869.83680,26583.870176
569,8334.58960,9946.465500
1034,17748.50620,24351.235531
198,5699.83750,7180.079483
...,...,...
1084,15817.98570,19328.995552
726,1964.78000,4231.575350
1132,4454.40265,5715.001346
725,38344.56600,40960.107948


In [8]:
import pickle 

In [9]:
filename = "model.pickle"
pickle.dump(Pol_reg, open(filename, "wb"))

In [10]:
for column in x.columns:
    print(column)

age
average_income
bmi
children
smoker_yes


In [11]:

pickle_out = open("model.pkl", "wb")
pickle.dump(Pol_reg, pickle_out)
pickle_out.close()